**SETUP**

In [ ]:
from openai import OpenAI
import os, json, itertools
from IPython.display import display, Markdown, Latex


In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('ai_agents_openai')

In [ ]:
# Set up OpenAI API Key in the os
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
client = OpenAI()

# Define the inputs

In [ ]:
MODEL = "gpt-4.1"
MODEL_MINI = "gpt-4.1-mini"
TOOLS = [{ "type": "web_search" }]
developer_message="""
You are an expert Deep Researcher.
You provide complete and in depth research to the user.
"""

In [ ]:
#Request the topic of research
topic = input("Please enter the research topic: \n")
topic

Please enter the research topic: 
I am buying a house 423 St Clair Court Bruce will this house afforable to me?


'I am buying a house 423 St Clair Court Bruce will this house afforable to me?'

In [ ]:
# Define here the prompt to clarify
prompt_to_clarify = f"""
Ask 5 numbered clarifying question to the user about the topic: {topic}.
The goal of the questions is to understand the intended purpose of the research.
Reply only with the questions.
"""

**LLM asks questions**

In [ ]:
clarify = client.responses.create(
    model= MODEL_MINI,
    input= prompt_to_clarify,
    instructions= developer_message
)

In [ ]:
questions = clarify.output[0].content[0].text.split( "\n")
questions

['1. What is your current monthly income and any other sources of financial support?  ',
 '2. What is your budget or maximum amount you are willing to spend on the house?  ',
 '3. Do you want an analysis that includes mortgage options, interest rates, and loan terms?  ',
 '4. Are you interested in understanding the additional costs such as property taxes, insurance, and maintenance for this house?  ',
 '5. Do you want a comparison of this house’s affordability relative to other similar properties in the Bruce area?']

In [ ]:
#Ask each question to the user.
answers = []
for question in questions:
  answer = input(question+"\n")
  answers.append(answer)

1. What is your current monthly income and any other sources of financial support?  
My annual gross income is 145k and I do not any other souce of financial support. I have investments of about 300k and I have 20% of down payment
2. What is your budget or maximum amount you are willing to spend on the house?  
1.5 million
3. Do you want an analysis that includes mortgage options, interest rates, and loan terms?  
yes
4. Are you interested in understanding the additional costs such as property taxes, insurance, and maintenance for this house?  
yes
5. Do you want a comparison of this house’s affordability relative to other similar properties in the Bruce area?
sure


In [ ]:
# Write the prompt_goals prompt
prompt_goals = f"""
Using the user answers {answers} to questions {questions}, write a goal sentence and 5 web search queries for the research about {topic}
Output: A json list of 5 web search queries and a goal sentence that wil reach it
Format: {{\"goal\":\"...\", \"queries\":[\"q1\",.....]}}
"""

In [ ]:
# Goal and Queries
goal_and_queries = client.responses.create(
    model= MODEL,
    input = prompt_goals,
    previous_response_id=clarify.id,
    instructions= developer_message
)

In [ ]:
import json
print(json.dumps(json.loads(goal_and_queries.output[0].content[0].text), indent=4))

{
    "goal": "To determine if purchasing 423 St Clair Court, Bruce is affordable based on your $145,000 annual income, $300,000 in investments, 20% down payment, and $1.5 million budget, including analysis of mortgage options, loan terms, additional ownership costs, and comparison to similar homes in the area.",
    "queries": [
        "Current listing price and details for 423 St Clair Court Bruce real estate",
        "Mortgage affordability calculator for $1.5 million home with $145k income and 20% down payment",
        "Property taxes, insurance, and average maintenance costs for homes in Bruce Ontario",
        "Recent sales prices and affordability of similar properties in Bruce Ontario",
        "Best mortgage rates and terms for 20% down payment in Bruce Ontario 2024"
    ]
}


In [ ]:
plan = json.loads(goal_and_queries.output[0].content[0].text)

In [ ]:
goal = plan["goal"]
queries = plan["queries"]

print(goal)
print(queries[0])

To determine if purchasing 423 St Clair Court, Bruce is affordable based on your $145,000 annual income, $300,000 in investments, 20% down payment, and $1.5 million budget, including analysis of mortgage options, loan terms, additional ownership costs, and comparison to similar homes in the area.
Current listing price and details for 423 St Clair Court Bruce real estate


**Web Searches**

In [ ]:
# Web search with the responses endpoint
web_search = client.responses.create(
    model=MODEL,
    input= f"seach: {queries[0]}",
    instructions= developer_message,
    tools = TOOLS
)

In [ ]:

web_search.output[1].id

'msg_68a62c64df6481929185e3a3bd88c9f307fa3de962d6abc1'

In [ ]:
web_search.output[1].content[0].text

'\n## [423 St Clair Court, Kincardine, ON](https://www.google.com/maps/search/423+St+Clair+Court%2C+Kincardine%2C+ON)\n\n\nThe property at 423 St Clair Court in Kincardine, Ontario, is currently listed for sale at $745,000. This raised bungalow-style detached home offers between 1,100 and 1,500 square feet of living space. It features three bedrooms on the upper level and two additional bedrooms on the lower level, totaling five bedrooms. The home includes two 4-piece bathrooms, one on each level. The main kitchen is located on the upper floor, complemented by a secondary kitchen on the lower level, suggesting potential for an in-law suite or rental opportunity. The property also boasts a finished basement with a recreation room measuring approximately 4.72 x 7.34 meters. Additional amenities include central air conditioning, a fireplace, and an attached two-car garage, providing a total of six parking spaces. The lot measures 44.42 feet in frontage and 112.66 feet in depth. As of 2024

In [ ]:
# Build the function for the web searches
def run_search(q):
  web_search = client.responses.create(
    model=MODEL,
    input= f"seach: {q}",
    instructions= developer_message,
    tools = TOOLS
  )
  return {"query": q,
          "resp_id": web_search.output[1].id,
          "research_output": web_search.output[1].content[0].text}

In [ ]:
run_search(queries[0])

{'query': 'Current listing price and details for 423 St Clair Court Bruce real estate',
 'resp_id': 'msg_68a62c9dac84819e80fb3552e40d7e9f0eca4488f986042a',
 'research_output': '\n## [423 St Clair Court, Kincardine, ON](https://www.google.com/maps/search/423+St+Clair+Court%2C+Kincardine%2C+ON)\n\n\nThe property at 423 St Clair Court in Kincardine, Ontario, is currently listed for sale at $745,000. ([zolo.ca](https://www.zolo.ca/kincardine-real-estate/423-st-clair-court?utm_source=openai)) This raised bungalow offers a total of five bedrooms and two bathrooms, with a finished basement that includes an additional kitchen, making it suitable for multi-generational living or rental opportunities.\n\n**Property Details:**\n\n- **Type:** Detached Raised Bungalow\n- **Size:** 1,100 to 1,500 square feet\n- **Bedrooms:** 3 on the upper level, 2 on the lower level\n- **Bathrooms:** 2 (one on each level)\n- **Kitchens:** 1 on the upper level, 1 on the lower level\n- **Basement:** Finished with an 

**Verifying if we have enough information to reach goal**

In [ ]:
# create a function that evaluates the outcome
def evaluate(collected):
  review = client.responses.create(
      model = MODEL,
      input = [
          {"role": "developer", "content": f"Reasearch goal: {goal}"},
          {"role": "assistant", "content": json.dumps(collected)},
          {"role": "user", "content": "Does this information will fully satisfy the goal? Answer Yes or No only!"}
      ],
      instructions= developer_message
  )
  return "yes" in review.output[0].content[0].text.lower()

In [ ]:
collected = []
for _ in itertools.count():
  for q in queries:
    print(f"Collecting data for query: {q}")
    collected.append(run_search(q))
  if evaluate(collected):
    break

  #If no: perform 5 more questions
  more_searches = client.responses.create(
      model = MODEL,
      input = [
          {"role": "assistant", "content": f"Current data: {json.dump(collected)}"},
          {"role": "developer", "content": f"Reasearch goal: {goal}. write 5 other web searchs to achieve the goal"},
      ],
      instructions= developer_message,
      previous_response_id=goal_and_queries.id
  )

  queries = json.loads(more_searches.output[0].content[0].text)

In [ ]:
print(collected)

[{'query': 'Current listing price and details for 423 St Clair Court Bruce real estate', 'resp_id': 'msg_68a62cb9cca08196828fb8092b0734f304b50cf426f83170', 'research_output': "\n## [423 St Clair Court, Kincardine, ON](https://www.google.com/maps/search/423+St+Clair+Court%2C+Kincardine%2C+ON)\n\n\nThe property at 423 St Clair Court in Kincardine, Ontario, is currently listed for sale at $745,000. ([zolo.ca](https://www.zolo.ca/kincardine-real-estate/423-st-clair-court?utm_source=openai)) This raised bungalow offers a total of five bedrooms and two bathrooms, with a finished basement that includes an additional kitchen, making it suitable for multi-generational living or rental opportunities.\n\n**Property Details:**\n\n- **Type:** Detached Raised Bungalow\n- **Size:** 1,100 to 1,500 square feet\n- **Bedrooms:** 3 on the upper level, 2 on the lower level\n- **Bathrooms:** 2 (one on each level)\n- **Kitchens:** 1 on the upper level, 1 on the lower level\n- **Basement:** Finished with an a

**Write the final report**

In [ ]:
# Use the responses endpoint to write the deep research report
report = client.responses.create(
    model=MODEL,
    input = [
        {"role": "developer", "content": (f"Write a complete and detailed report about reasearch goal: {goal}"
        "Cite sources inline using  [n] and append a reference"
        "list mapping [n] to url")},
        {"role": "assistant", "content": json.dumps(collected)},
    ],
    instructions= developer_message
)

print("\n=== FINAL REPORT ===")
display(Markdown(report.output[0].content[0].text))



=== FINAL REPORT ===


# Research Report: Affordability Analysis for Purchasing 423 St Clair Court, Bruce, Ontario

## Executive Summary

This report evaluates the affordability of purchasing 423 St Clair Court, Bruce, based on a projected household income of $145,000/year, $300,000 in investments, a 20% down payment, and an upper budget of $1.5 million. It includes comprehensive analysis of the subject property, mortgage options and terms, ongoing ownership costs, and a comparative evaluation of similar homes. The assessment shows that the chosen property is in line with your finances, but properties at the upper end of your budget may risk financial strain given current mortgage rates.

---

## 1. Property Overview: 423 St Clair Court

- **Listing Price:** $745,000
- **Type:** Detached raised bungalow
- **Size:** 1,100–1,500 sq ft
- **Bedrooms:** 5 (3 upper, 2 lower)
- **Bathrooms:** 2
- **Features:** Finished in-law basement apartment, double garage, quiet cul-de-sac
- **2024 Property Taxes:** $4,563.64
- **Neighborhood:** Kincardine; close to schools, local businesses; above average local market value; average nearby home value: $687,788 ([1](https://www.zolo.ca/kincardine-real-estate/423-st-clair-court?utm_source=openai))

---

## 2. Affordability Analysis

### 2.1 Down Payment and Mortgage Needs

For a $745,000 property with a 20% down payment:
- **Down Payment:** $149,000 (drawn from investments)
- **Mortgage Required:** $596,000

### 2.2 Mortgage Options and Terms

Current Ontario rates for well-qualified buyers (July 2024–2025):
- **5-Year Fixed:** ~4.24%
- **5-Year Variable:** ~5.35%
- **Amortization:** 25–30 years (no insurance required at 20%+ down) ([5](https://themortgagefirm.ca/rates?utm_source=openai))

#### Estimated Monthly Mortgage Payments
(Assuming a 25-year amortization, 4.24% fixed, $596,000 principal):

- **Monthly Payment:** ≈ $3,209

### 2.3 Additional Ownership Costs

#### Property Taxes
- **Annually for 423 St Clair:** $4,564 → $380/month ([1](https://www.zolo.ca/kincardine-real-estate/423-st-clair-court?utm_source=openai))

#### Home Insurance
- **Estimated Annual Premium:** $800–$1,200 → $67–$100/month ([3](https://www.southbruce.ca/en/living-here/tax-rates.aspx?utm_source=openai))

#### Maintenance
- **Budget:** 1–3% of property value/year → $7,450–$22,350/year → $620–$1,860/month ([3](https://www.southbruce.ca/en/living-here/tax-rates.aspx?utm_source=openai))

#### Utilities & Miscellaneous
- **Average Monthly:** $300–$400 (electricity, heat, water, waste)

#### TOTAL ESTIMATED MONTHLY COSTS
| Expense           | Amount (Monthly) |
|-------------------|-----------------|
| Mortgage          | $3,209          |
| Property Taxes    | $380            |
| Insurance         | $84 (midpoint)  |
| Maintenance Avg.  | $1,240 (midpoint)|
| Utilities         | $350            |
| **Total**         | **$5,263**      |

---

## 3. Affordability Relative to Income

**Annual Income:** $145,000 → $12,083/month before tax

### Debt-to-Income Calculations

- **“Safe” guideline:** Housing expenses ≤ 28–30% of gross monthly income ([2](https://www.omnicalculator.com/finance/home-affordability?utm_source=openai))
- **Calculated Ratio:** $5,263 / $12,083 ≈ 44%
- **Implication:** While this is above conservative standards, you may still qualify with a favorable credit profile and low other debts.

### $1.5M Budget Scenario

- **Down Payment:** $300,000
- **Mortgage Required:** $1,200,000
- **Monthly Payment @ 4.24% (25 yrs):** ≈ $6,482
- **Total Housing Expenses (including taxes and insurance):** $9,933/month ([2](https://www.omnicalculator.com/finance/home-affordability?utm_source=openai))
- **DTI Ratio:** $9,933 / $12,083 ≈ 82% (well above prudent levels; high risk)

**Conclusion:** While the 423 St Clair Court is affordable at $745,000, homes approaching $1.5 million are likely out of reach for comfortable ownership on a $145,000 income ([2](https://www.omnicalculator.com/finance/home-affordability?utm_source=openai)).

---

## 4. Area Market Comparison

### Recent Comparable Sales

- **Median Sale Price - Bruce County:** $594,000–$687,000 ([4](https://www.zolo.ca/northern-bruce-peninsula-real-estate/sold?utm_source=openai))
- **Recent Comps:**
  - 105 Golden Pond Dr: $920,000 (4bd/2ba, 1,709 sq.ft.)
  - 225 Devinwood Ave: $800,000 (3bd/3ba, 1,500 sq.ft.)
  - 26 Thomas St: $542,000 (3bd/2ba, 1,167 sq.ft.)
- **423 St Clair Position:** Priced above typical county transaction, but justified by size, condition, and rental potential.

---

## 5. Mortgage Product Recommendations

- Choose a 5-year fixed rate for stability at current historical lows (~4.24%), considering payment predictability.
- Amortize over 25 years to build equity faster; consider 30 years only to reduce monthly payments if desired ([5](https://themortgagefirm.ca/rates?utm_source=openai)).
- Mortgage brokers may have access to better deals than banks ([5](https://themortgagefirm.ca/rates?utm_source=openai)).
- Pre-approval essential to lock in rate and demonstrate credibility ([5](https://themortgagefirm.ca/rates?utm_source=openai)).

---

## 6. Risks and Additional Considerations

- Interest rate risk: Future renewals may see higher rates post-2029.
- Maintenance: Bungalows with secondary suites typically have greater repair and utility costs.  
- Liquidity: Using a significant portion of your $300,000 investments for a down payment reduces liquid reserves; keep contingency for emergencies.

---

## 7. Summary and Conclusion

- **423 St Clair Court at $745,000 is feasible**, with affordable monthly housing costs relative to your income, especially if you’re disciplined with other spending.
- **Stretching towards $1.5 million** would be financially imprudent without significantly higher income or larger down payment.
- Local market trends suggest the price is above average for the area, but the property’s size and two-unit layout (potential rental) support the ask.
- **Best practice**: Maintain housing costs below 40% of gross income, reserve emergency funds, and favor fixed-rate mortgage products.

---

## 8. References
1. [Zolo.ca Property Listing – 423 St Clair Court](https://www.zolo.ca/kincardine-real-estate/423-st-clair-court?utm_source=openai)
2. [Home Affordability Calculator – Omni Calculator](https://www.omnicalculator.com/finance/home-affordability?utm_source=openai)
3. [South Bruce Tax Rates](https://www.southbruce.ca/en/living-here/tax-rates.aspx?utm_source=openai)
4. [Zillow/Zolo Bruce County Recent Sales](https://www.zillow.com/bruce-county-on/sold/?utm_source=openai), [Zolo Northern Bruce Peninsula](https://www.zolo.ca/northern-bruce-peninsula-real-estate/sold?utm_source=openai)
5. [The Mortgage Firm: Mortgage Rates and Tips in Ontario 2025](https://themortgagefirm.ca/rates?utm_source=openai)

---

**Final Recommendation:** If you focus on properties in the $700,000–$800,000 range, including 423 St Clair Court, the purchase is comfortably affordable for your financial profile, offering stability, security, and flexibility for the future.